In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r /content/drive/MyDrive/vignesh/ps/colab/* /content/

In [ ]:
!unzip -q '/content/drive/MyDrive/vignesh/ps/syn_data.zip'

In [ ]:
#!pip3 install tensorflow==2.4.1

In [ ]:
!pip3 install albumentations==0.4.2

In [ ]:
#%matplotlib inline
import cv2
import tensorflow as tf
print(tf.__version__)

import pandas as pd
from glob import glob
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import * 
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
from matplotlib import patches
import albumentations as A
from albumentations import BboxParams
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import shutil
import numpy as np
from yolotxt2rectangle import yolotxt2rectangle
from tensorflow.keras.applications import MobileNetV3Large

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
#globals 
# n_classes is 1 higher than YOLO since class  0 is background in SSD
size=(784,784,3)
colormode='rgb'
normalize_factor=1/255.
width=size[0]
height=size[1]
channels=size[2]
batch_size=16
grid_size_w=96.0
grid_size_h=96.0
grid_factor_w=1/grid_size_w
grid_factor_h=1/grid_size_h
with open('classes.txt','r') as f:
    data=f.read()
LABELS=[i for i in data.split('\n') if len(i)>0]
nclasses=len(LABELS)
print('num classes:',nclasses)

In [ ]:
train_fnames=sorted(glob('data/ab_full/train/*.txt'))+sorted(glob('data/ab_full/upsample/*.txt'))
val_fnames=sorted(glob('data/ab_full/test/*.txt'))
test_fnames=val_fnames
print('Num train files:',len(train_fnames))
print('Num val files:',len(val_fnames))

In [ ]:
class datagen(Sequence):

  def __init__(self,train_im_path,augmentations):
    self.batch_size=batch_size
    self.train_im_paths=train_im_path
    self.img_size=size
    print(self.img_size)
    self.nchannels=channels
    self.shuffle=shuffle
    self.augmentations=augmentations
    self.grid_size_w=grid_size_w
    self.grid_size_h=grid_size_h
    self.info=5
    if self.augmentations!=None:
      print('Augs used:',self.augmentations)
    self.on_epoch_end()
    print('Num images found : {}'.format(len(self.train_im_paths)))

  def __len__(self):
    return int(np.ceil(len(self.train_im_paths) / self.batch_size))

  def __getitem__(self,index):
    if self.augmentations!=None:
      self.aug=self.get_aug(self.augmentations)
    indexes = self.indexes[index*self.batch_size:min((index+1)*self.batch_size,len(self.train_im_paths))]

    list_IDs_im = [self.train_im_paths[k] for k in indexes]
    
    try:
      X,y= self.data_generation(list_IDs_im,colormode)
    except ValueError:
      #print(list_IDs_im,)
      print('problem averted')
      X,y=self.prev_X,self.prev_Y

    self.prev_X,self.prev_Y=X,y
    return X,y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.train_im_paths))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def get_aug(self,aug, min_area=0., min_visibility=0.):
    return A.Compose(aug, A.BboxParams(format='pascal_voc', min_area=min_area, 
                                       min_visibility=min_visibility, label_fields=['category_id']))
  
  def data_generation(self,list_IDs,color_mode):
    

    X = []

    Y = np.zeros((len(list_IDs),int(self.grid_size_h),int(self.grid_size_w), 1, nclasses + self.info))

    for count,i in enumerate(list_IDs):
      #print('filename:',i)
      self.curr_name=i
      self.flag=False
      
    
      if os.path.exists(i[:-4]+'.jpg'):
        x=cv2.resize(cv2.imread(i[:-4]+'.jpg'),(width,height),interpolation=cv2.INTER_AREA)
        x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)

      if os.path.exists(i[:-4]+'.png'):
        x=cv2.resize(cv2.imread(i[:-4]+'.png'),(width,height),interpolation=cv2.INTER_AREA)
        x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)

       
      #print(box)
      box=yolotxt2rectangle(i,y_height=height,x_height=width,last_class=0)
      self.curr_box=box


      if self.augmentations!=None:

       
        annotations = {'image': x, 'bboxes': [i for i in box[:,1:]], 'category_id': [i for i in box[:,0]]}
        augmented = self.aug(**annotations)
        x=augmented['image']

        box_array=np.empty(shape=(len(augmented['bboxes']),5))
        box_array[:,1:]=np.array(augmented['bboxes'])
        box_array[:,0]=np.array(augmented['category_id'])
        box=box_array.copy()

      x=np.expand_dims(x,axis=0)
      self.curr_frame=x
      #print(x.shape)
      X.append(x)
        
      #print('Num boxes:',len(box))
        
      coords=[]

      
      for label, bndbox in zip(box[:,0],box[:,1:]):
          xmin = int(bndbox[0])
          ymin = int(bndbox[1])

          xmax = int(bndbox[2])
          ymax = int(bndbox[3])

          w = (xmax - xmin) / width
          h = (ymax - ymin) / height

          cx = ((xmax + xmin) / 2) / width
          cy = ((ymax + ymin) / 2) / height

          

          cx = cx * self.grid_size_w
          cy = cy * self.grid_size_h
            
          if (int(cy), int(cx),0) in coords:
            print('Duplicate:',(int(cy), int(cx)),i)  
            plt.imshow(x.squeeze())
            plt.show()
                
          else:
            
            Y[count,int(cy), int(cx), 0, 0] = 1 #objectness
            Y[count,int(cy), int(cx), 0, 1] = cx - int(cx)
            Y[count,int(cy), int(cx), 0, 2] = cy - int(cy)
            Y[count,int(cy), int(cx), 0, 3] = w
            Y[count,int(cy), int(cx), 0, 4] = h
        
            coords.append((int(cy), int(cx),0))

          

          class_index = int(label)
          Y[count,int(cy), int(cx), 0, 5 + class_index] = 1.0

    
    X=np.array(X).squeeze()

    return X*normalize_factor,Y
  



In [ ]:
!mkdir encoded

In [ ]:
augmentations=[
               A.ShiftScaleRotate(rotate_limit=5,p=0.3,border_mode=cv2.BORDER_CONSTANT,mask_value=0,scale_limit=0.05,shift_limit=0.05),
               A.RandomBrightnessContrast(p=0.2),
               A.HorizontalFlip(p=0.3),
               A.VerticalFlip(p=0.3),
               A.RandomSunFlare(p=0.3,src_radius=150,flare_roi=(0, 0, 1,1),src_color=(224,224,224)),
               A.ChannelShuffle(p=0.3)
               ] 
augmentations=None
val_augmentations=None

In [ ]:
#train_im_path,augmentations,batch_size,nclasses,num_boxes,predictor_sizes,img_size=256,nchannels=3,normalize_img=False,shuffle=True
gen=datagen(shuffle(train_fnames),None)
aug_gen=datagen(shuffle(train_fnames),augmentations)
valgen=datagen(val_fnames,val_augmentations)
testgen=datagen(val_fnames,val_augmentations)

In [ ]:
for id_ in range(len(aug_gen)):

  #id_=np.random.randint(0,len(aug_gen))
  # #id_=18
  #print(id_)


  img,arr=aug_gen[id_]
#   grid_w=arr.shape[1]
#   grid_h=arr.shape[2]
#   #print(img.shape,arr.shape)
#   for batch in range(len(arr)):
#     num_boxes=0
#     for k in range(1):
#         for i in range(grid_w):
#           for j in range(grid_h):
#             confidence = arr[batch,i, j, k, 0].squeeze()



#             if confidence==1.0:
#               num_boxes+=1
#               cx,cy,w,h=arr[batch,i, j, k,1:5]

#               cx = ((j + (cx)) / grid_size_w)
#               cy = ((i + (cy)) / grid_size_h)

#               cx=cx*width
#               cy=cy*height
#               w=w*width
#               h=h*height

#               xmin=int(cx-(w/2))
#               ymin=int(cy-(h/2))
#               xmax=int(cx+(w/2))
#               ymax=int(cy+(h/2))


#               text=LABELS[int(np.argmax(arr[batch,i, j, 0, 5:]))]

#               cv2.rectangle(img[batch], (int(xmin),int(ymin)), (int(xmax),int(ymax)), (0,0,0), 2)
#               #cv2.putText(img[batch], text, (int(cx) - 10, int(cy) - 10),cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1)
    

#     plt.imshow(img[batch])
#     plt.show()
#   #329881_1_4787779

# #Duplicate: (67, 51) data/ab_full/midv_train/DG16_17.txt
#20201203_171818_PASSPORT_5fc8bb6acdb35456492104b6_d37b6358-004b-4f78-a315-507ca7f1427f_frontImage_1_1347339.txt

In [ ]:
for id_ in range(len(valgen)):

  #id_=np.random.randint(0,len(aug_gen))
  # #id_=18
  #print(id_)


  img,arr=valgen[id_]

In [ ]:
# id_=np.random.randint(0,len(valgen))
# #id_=211
# print(id_)
# img,arr=valgen[id_]
# grid_w=arr.shape[1]
# grid_h=arr.shape[2]
# print(img.shape,arr.shape)
# for batch in range(len(arr)):

#   for i in range(grid_w):
#     for j in range(grid_h):
#       confidence = arr[batch,i, j, 0, 0].squeeze()

#       if confidence==1.0:
#         cx,cy,w,h=arr[batch,i, j, 0,1:5]
        
#         cx = ((j + (cx)) / grid_size_w)
#         cy = ((i + (cy)) / grid_size_h)

#         cx=cx*width
#         cy=cy*height
#         w=w*width
#         h=h*height

        

#         xmin=int(cx-(w/2))
#         ymin=int(cy-(h/2))
#         xmax=int(cx+(w/2))
#         ymax=int(cy+(h/2))


#         text=LABELS[int(np.argmax(arr[batch,i, j, 0, 5:]))]

#         cv2.rectangle(img[batch], (int(xmin),int(ymin)), (int(xmax),int(ymax)), (0,0,0), 2)
#         cv2.putText(img[batch], text, (int(cx) - 10, int(cy) - 10),cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1)
  
  
#   #print(img[batch].shape)
#   if colormode=="grayscale":
#     plt.imshow(img[batch],cmap='gray')
#   else:
#     plt.imshow(img[batch])
#   plt.show()

In [ ]:
img.shape,arr.shape,img.max(),normalize_factor

In [ ]:
def multiboxes_loss(y_true, y_pred):
    l_coords = 5.0
    l_noob = 0.5
    coords = y_true[:, :, :, :, 0] * l_coords

    noobs = -1 * (y_true[:, :, :, :, 0] - 1) * l_noob
    p_pred = y_pred[:, :, :, :, 0]
    p_true = y_true[:, :, :, :, 0]
    x_true = y_true[:, :, :, :, 1]
    x_pred = y_pred[:, :, :, :, 1]
    yy_true = y_true[:, :, :, :, 2]
    yy_pred = y_pred[:, :, :, :, 2]
    w_true = y_true[:, :, :, :, 3]
    w_pred = y_pred[:, :, :, :, 3]
    h_true = y_true[:, :, :, :, 4]
    h_pred = y_pred[:, :, :, :, 4]

    cl_true = y_true[:, :, :, :, 5:]
    cl_pred = y_pred[:, :, :, :, 5:]

    p_loss_absent = K.sum(K.square(p_pred - p_true) * noobs)
    p_loss_present = K.sum(K.square(p_pred - p_true))
    x_loss = K.sum(K.square(x_pred - x_true) * coords)
    yy_loss = K.sum(K.square(yy_pred - yy_true) * coords)
    xy_loss = x_loss + yy_loss
    w_loss = K.sum(K.square(K.sqrt(w_pred) - K.sqrt(w_true)) * coords)
    h_loss = K.sum(K.square(K.sqrt(h_pred) - K.sqrt(h_true)) * coords)
    wh_loss = w_loss + h_loss

    cl_loss = K.sum(K.square(cl_true - cl_pred))
    loss = p_loss_absent + p_loss_present + xy_loss + wh_loss + cl_loss

    return loss

In [ ]:
def yolo_model(input_shape, classes, info=5):
    K.clear_session()
    inp = Input(input_shape)

    
    base_model = MobileNetV3Large(input_tensor=inp,
                                  include_top=False, 
                                  weights='imagenet'
                                  )
    
    for layer in base_model.layers:
      layer.trainable=True
    

    #fe_layer = base_model.layers[-7].output

    fe_layer = base_model.get_layer('multiply_13').output


    conv = Conv2D(512, (3, 3), activation="relu", padding="same")(fe_layer)
    conv = BatchNormalization()(conv)
    conv = LeakyReLU(alpha=0.275)(conv)
    conv= UpSampling2D()(conv)

    conv = Conv2D(128, (3, 3), activation="relu", padding="same")(conv)
    conv = BatchNormalization()(conv)
    conv = LeakyReLU(alpha=0.275)(conv)
    conv= UpSampling2D()(conv)
    
    conv = Conv2D(128, (3, 3), activation="relu", padding="same")(conv)
    conv = BatchNormalization()(conv)
    conv = LeakyReLU(alpha=0.275)(conv)
    #conv= UpSampling2D()(conv)

    conv = Conv2D(classes + info, (3, 3), activation="sigmoid", padding="same")(conv)

    output = Reshape((int(grid_size_h),int(grid_size_w), 1, classes + info))(conv)
    model = Model(inp, output)

    return model

model=yolo_model(size,nclasses)
# model=load_model('/content/drive/MyDrive/vignesh/field_detection/passport/deu/models/deupassportv1_yolo_90mAP42.0_acc100_wo.h5',custom_objects={
#     'multiboxes_loss':multiboxes_loss
# })
print(model.summary())

In [ ]:
model.compile(loss=multiboxes_loss,optimizer='adam')

In [ ]:
class mAP(tf.keras.callbacks.Callback):
    
    def __init__(self):
        super(mAP, self).__init__()
        self.num_accurate=None
        self.current_epoch=None
        self.img_width=width
        self.img_height=height
        self.frame_width=None
        self.frame_height=None
        self.color_mode=colormode
        
    def on_train_begin(self, logs=None):
        self.nb_epoch = self.params['epochs']
        
        
    def accumulate(self,filepath):

      if os.path.exists(filepath[:-4]+'.jpg'):
        src_img=cv2.imread(filepath[:-4]+'.jpg')
      if os.path.exists(filepath[:-4]+'.png'):
        src_img=cv2.imread(filepath[:-4]+'.png')
      self.frame_width,self.frame_height=src_img.shape[1],src_img.shape[0]

      img=cv2.resize(src_img,(width,height),interpolation=cv2.INTER_AREA)

      if self.color_mode=="grayscale":
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=np.expand_dims(img,axis=-1)

      else:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

      blob=np.expand_dims(img,axis=0)*normalize_factor
      iou_threshold=0.5
      confidence_thresh=0.5

      arr=model.predict(blob)


      assert(os.path.exists('gt')),'No ground truth folder found'
      shutil.copy(filepath[:-4]+'.txt','gt')
      assert(os.path.exists('det')),'No detection folder found'


      valid_boxes=[]


      for batch in range(len(arr)):
        
        for i in range(grid_w):
          
          for j in range(grid_h):

            confidence = arr[batch,i, j, 0, 0].squeeze()

            if confidence>=0.25:
              valid_boxes.append(arr[batch,i, j, 0,:])
              cx,cy,w,h=arr[batch,i, j, 0,1:5]
              
              cx = ((j + (cx)) / grid_size)
              cy = ((i + (cy)) / grid_size)

              class_index=int(np.argmax(arr[batch,i, j, 0, 5:]))


              with open('det/{}.txt'.format(filepath.split('/')[-1][:-4]),'a')as f:
                
                f.write(f'{class_index} {confidence} {cx} {cy} {w} {h} \n')

        gt=np.loadtxt(filepath).squeeze()

        if len(gt.shape)<2:
          gt=np.expand_dims(gt,axis=0)
        
        if len(valid_boxes)==len(gt):
          self.num_accurate+=1


      
        
    
    def compute_mAP(self,filedir,reset=False):
      if reset:
        os.system('rm -r gt')
        os.system('rm -r det')
        os.system('mkdir gt')
        os.system('mkdir det')


      print('Num files:',len(val_fnames))
      for file in val_fnames:
          self.accumulate(file)
          
       
      os.system('python3 pascalvoc.py -gt gt -det det -gtcoords rel -detcoords rel -t 0.9 -gtformat xywh -detformat xywh -imgsize {},{} -np'.format(width,height))
      with open('results/results.txt','r') as f:
          f=f.read()
          print(f.split('\n')[-3:])
          return f.split('\n')[-3].strip('mAP: %')
          #return f.split('\n')[-3].strip('mAP: %'),f.split('\n')[7].strip('AP: %'),f.split('\n')[32].strip('AP: %')


        
    def on_test_end(self, batch, logs=None):
      # if self.current_epoch<10:
      #   return
      self.num_accurate=0
      #print('val mAP:',self.compute_mAP('/content/val',reset=True))
      testmAP=self.compute_mAP('test',reset=True)
      current_accuracy=round((self.num_accurate/len(val_fnames))*100)
      model_name='/content/drive/MyDrive/vignesh/ps/models/mrz_yolo_90mAP{}_acc{}.h5'.format(testmAP,current_accuracy)
           
      #float(testmAP)>=60 and 

      if float(testmAP)>=1 and current_accuracy>=99:
        model.save(model_name.replace('.h5','_wo.h5'))
        model.save(model_name,include_optimizer=False)
      
      print('test mAP:',testmAP)
      print(f'Accuracy:{current_accuracy} samples_correct:{self.num_accurate}/{len(val_fnames)}')
      self.num_accurate=0

    def on_epoch_end(self, epoch, logs=None):

      self.current_epoch=epoch
      




In [ ]:
model.optimizer.lr

In [ ]:
K.set_value(model.optimizer.lr,5e-4)

In [ ]:
initial_epoch=5
final_epoch=2000
print('lr:',K.eval(model.optimizer.lr))
print('augs used:',augmentations)
print('val_augs used:',val_augmentations)
print('num train samples:',len(aug_gen)*batch_size)
history=model.fit_generator(aug_gen,steps_per_epoch=len(aug_gen),
                            epochs=final_epoch,
                            initial_epoch=initial_epoch,
                            validation_data=valgen,
                            validation_steps=len(valgen),
                            callbacks=None,
                            #callbacks=[mAP()],
                            verbose=1)

In [ ]:
model.evaluate_generator(valgen,
                        callbacks=[mAP()],
                         verbose=1,steps=len(valgen))

In [ ]:
model.save('/content/drive/MyDrive/vignesh/field_detection/idcard_reduced/countrywise/uaeid/models/uaeid_yolo_temp_wo.h5',include_optimizer=True)

In [ ]:
for name in shuffle(test_fnames):
  print(name)
  img=cv2.imread(name.replace('txt','jpg'))
  if colormode=='grayscale':
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=np.expand_dims(img,axis=-1)
  else:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  blob=cv2.resize(img,(width,height),interpolation=cv2.INTER_AREA)

  blob=np.expand_dims(blob,axis=0)*normalize_factor
  arr=model.predict(blob)

  valid_classes=[]


  for batch in range(len(arr)):
    
    for i in range(grid_w):
      
      for j in range(grid_h):

        confidence = arr[batch,i, j, 0, 0].squeeze()

        if confidence>=0.5:
          cx,cy,w,h=arr[batch,i, j, 0,1:5]
          valid_classes.append(arr[batch,i, j, 0,1:5])
          
          cx = ((j + (cx)) / grid_size)
          cy = ((i + (cy)) / grid_size)

          cx=cx*img.shape[1]
          cy=cy*img.shape[0]
          w=w*img.shape[1]
          h=h*img.shape[0]

          

          xmin=int(cx-(w/2))
          ymin=int(cy-(h/2))
          xmax=int(cx+(w/2))
          ymax=int(cy+(h/2))


          text=LABELS[int(np.argmax(arr[batch,i, j, 0, 5:]))]

          cv2.rectangle(img, (int(xmin),int(ymin)), (int(xmax),int(ymax)), (0,0,255), 2)
          cv2.putText(img, text, (int(cx) - 10, int(cy) - 10),cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1)

  if len(valid_classes)==len(np.loadtxt(name).squeeze()):
    pass

  if colormode=="grayscale":
    plt.imshow(img.squeeze(),cmap='gray')
  else:
    plt.imshow(img)
  plt.show()

In [ ]:
for name in shuffle(train_fnames)[:10]:
  
  img=cv2.imread(name.replace('txt','jpg'))
  if colormode=='grayscale':
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=np.expand_dims(img,axis=-1)
  else:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  blob=cv2.resize(img,(width,height),interpolation=cv2.INTER_AREA)

  blob=np.expand_dims(blob,axis=0)*normalize_factor
  arr=model.predict(blob)

  valid_classes=[]


  for batch in range(len(arr)):
    
    for i in range(grid_w):
      
      for j in range(grid_h):

        confidence = arr[batch,i, j, 0, 0].squeeze()

        if confidence>=0.5:
          cx,cy,w,h=arr[batch,i, j, 0,1:5]
          valid_classes.append(arr[batch,i, j, 0,1:5])
          
          cx = ((j + (cx)) / grid_size)
          cy = ((i + (cy)) / grid_size)

          cx=cx*img.shape[1]
          cy=cy*img.shape[0]
          w=w*img.shape[1]
          h=h*img.shape[0]

          

          xmin=int(cx-(w/2))
          ymin=int(cy-(h/2))
          xmax=int(cx+(w/2))
          ymax=int(cy+(h/2))


          text=LABELS[int(np.argmax(arr[batch,i, j, 0, 5:]))]

          print(xmin,ymin,xmax,ymax)

          cv2.rectangle(img, (int(xmin),int(ymin)), (int(xmax),int(ymax)), (0,0,255), 2)
          cv2.putText(img, text, (int(cx) - 10, int(cy) - 10),cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1)

  gt=np.loadtxt(name).squeeze()

  if len(gt.shape)<2:
    gt=np.expand_dims(gt,axis=0)

  if len(valid_classes)==len(gt):
    continue
  #print(name,np.loadtxt(name).squeeze())
  if colormode=="grayscale":
    plt.imshow(img.squeeze(),cmap='gray')
  else:
    plt.imshow(img)
  plt.show()